### Dataset info
- Number of images: 20,580
- Number of labels: 120 (need 7 nodes to represent in binary)
- Files list: the names of the files (with jpg)
- Annotation list: the name of the files (without jpg)
- Labels: image labels (1-120)

Need to get all images to be the same size:
- Lossy: crop or downsample
- Additive: add whitespace around image

Misclassifed:
n02095314_80: wire fox terrier -> irish terrier

In [172]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
import imageio
import os
import bs4

%matplotlib inline

In [173]:
REPO_DIR = '/Users/jampp/workspace/breed-all-about-it/'
files_mat = sio.loadmat(os.path.join(REPO_DIR, 'data', 'lists', 'file_list.mat'))

In [199]:
# annotations
annotations = np.zeros(files_mat['annotation_list'].shape[0], dtype='<U100')
for i, annotation in enumerate(files_mat['annotation_list'][:,0]):
    annotations[i] = annotation[0]
    
# file names
files = np.zeros(files_mat['file_list'].shape[0], dtype='<U100')
for i, file in enumerate(files_mat['file_list'][:,0]):
    files[i] = file[0]
    
# array of labels
labels = files_mat['labels'][:,0]

In [224]:
np.squeeze?

In [225]:
def crop_images(files, annotations, directory): 
    """
    Crop image to the specifications given in the annotations.
    
    Parameters:
    -----------
    files
    annotations
    
    Returns:
    --------
    None
    """
    for i in range(13673, len(files)):
        original_image = imageio.imread(os.path.join(REPO_DIR, 'data', 'Images', files[i]))
        annotation = annotations[i]
        f = open(os.path.join(REPO_DIR, 'data', 'Annotation', annotations[i]), 'r')
        soup = bs4.BeautifulSoup(f, 'lxml')
        xmin = int(soup.bndbox.xmin.contents[0])
        xmax = int(soup.bndbox.xmax.contents[0])
        ymin = int(soup.bndbox.ymin.contents[0])
        ymax = int(soup.bndbox.ymax.contents[0])
        cropped_image = original_image[ymin:ymax, xmin:xmax, :3]
        os.makedirs(os.path.dirname(os.path.join(directory, 'data', 'Crops', files[i])), exist_ok=True)
        try:
            imageio.imwrite(os.path.join(directory, 'data', 'Crops', files[i]), cropped_image)
        except OSError:
            print(i)

In [226]:
# crop_images(files, annotations, REPO_DIR)

In [ ]:
# Next step: resize images